<a href="https://colab.research.google.com/github/ilya-lykov/google_colab_labs/blob/main/1_lab/1_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark ##Устаналвиваем pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=b131931bf5685c3f38d30934a497053ffd8ba562091ffe50041853eb43a92938
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession ##Необходимые импорты
import pyspark.sql.functions as F

In [ ]:
spark = (SparkSession.builder.appName("Test").getOrCreate()) ##Создаем инстанс спарка

In [ ]:
acc = spark.read.csv("/content/drive/MyDrive/University/data/2015/DfTRoadSafety_Accidents_2015.csv", header=True, inferSchema=True)
##Прочитал файл из своего Google Хранилища
acc1 = acc.select("Accident_Index","Date", "Number_of_Vehicles", "Weather_Conditions")
##Сделал выборку по столбцам
acc1.show()

+--------------+----------+------------------+------------------+
|Accident_Index|      Date|Number_of_Vehicles|Weather_Conditions|
+--------------+----------+------------------+------------------+
| 201501BS70001|12/01/2015|                 1|                 1|
| 201501BS70002|12/01/2015|                 1|                 1|
| 201501BS70004|12/01/2015|                 1|                 2|
| 201501BS70005|13/01/2015|                 1|                 1|
| 201501BS70008|09/01/2015|                 2|                 2|
| 201501BS70009|15/01/2015|                 2|                 1|
| 201501BS70010|15/01/2015|                 2|                 8|
| 201501BS70011|18/01/2015|                 2|                 1|
| 201501BS70012|16/01/2015|                 2|                 1|
| 201501BS70013|20/01/2015|                 2|                 8|
| 201501BS70014|23/01/2015|                 2|                 2|
| 201501BS70015|22/01/2015|                 1|                 1|
| 201501BS

In [ ]:
# Среднее количество транспортных средств для каждого типа погодных условий
average_vehicles = (
    acc1.groupBy("Weather_Conditions")
    .agg(F.avg("Number_of_Vehicles").alias("average_vehicles"))
)

# Вывод результата
average_vehicles.show()

+------------------+------------------+
|Weather_Conditions|  average_vehicles|
+------------------+------------------+
|                 1|1.8543801768917592|
|                 6|1.7485714285714287|
|                 3|1.7037037037037037|
|                 5|1.7269653423499578|
|                 9| 1.740080971659919|
|                 4|1.7851380042462845|
|                 8|1.7719033232628398|
|                 7|1.8338368580060422|
|                 2|1.7997943973271653|
+------------------+------------------+



In [ ]:
# Фильтрация данных по условиям
filtered_data = acc1.filter(
    (F.col("Weather_Conditions") == "3") &
    (F.col("Number_of_Vehicles") > 2)
)

# Показать результат
filtered_data.show()

+--------------+----------+------------------+------------------+
|Accident_Index|      Date|Number_of_Vehicles|Weather_Conditions|
+--------------+----------+------------------+------------------+
| 2015030000076|13/01/2015|                 3|                 3|
| 2015030000133|14/01/2015|                 3|                 3|
| 2015030000665|02/03/2015|                 3|                 3|
| 201504ED15021|14/01/2015|                 3|                 3|
| 201504ED15023|17/01/2015|                 4|                 3|
| 201504EG15009|28/01/2015|                 3|                 3|
| 201504EG15011|29/01/2015|                 3|                 3|
| 201504SA15022|13/01/2015|                 3|                 3|
| 2015051500172|17/01/2015|                 4|                 3|
| 2015051502780|07/11/2015|                 4|                 3|
| 201506K096301|17/01/2015|                 3|                 3|
| 201506K096323|17/01/2015|                 3|                 3|
| 201506P0

In [ ]:
# Преобразуем Date из строки в формат даты
acc1 = acc1.withColumn("      Date", F.to_date(F.col("Date"), "dd/MM/yyyy"))
#Делаем агрегацию по Дате
agg_by_date = (acc1
               .groupBy(F.col("Accident_Index"), F.col("Number_of_Vehicles"), F.col("Weather_Conditions"))
               .agg(F.min(F.col("Date").alias("min_date")))
               .orderBy(F.col("min(Date AS min_date)").asc())
               )

agg_by_date.show()


+--------------+------------------+------------------+---------------------+
|Accident_Index|Number_of_Vehicles|Weather_Conditions|min(Date AS min_date)|
+--------------+------------------+------------------+---------------------+
| 201501YR90315|                 2|                 1|           01/01/2015|
| 201501QK50060|                 4|                 1|           01/01/2015|
| 2015170H10015|                 2|                 4|           01/01/2015|
| 2015132110547|                 1|                 4|           01/01/2015|
| 201501ZT80005|                 2|                 2|           01/01/2015|
| 201501SX20044|                 2|                 2|           01/01/2015|
| 201501EO40016|                 1|                 1|           01/01/2015|
| 201501MM70020|                 1|                 1|           01/01/2015|
| 201504ED15004|                 2|                 2|           01/01/2015|
| 201501YR90704|                 2|                 9|           01/01/2015|